<a href="https://colab.research.google.com/github/tsakailab/cisexpkit/blob/master/Experiment/colab/passive_stereobm_MB05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 受動的ステレオの実践

## [ブロックマッチング](https://en.wikipedia.org/wiki/Block-matching_algorithm)で[視差](https://github.com/tsakailab/cisexpkit/blob/master/Experiment/Document/ActiveStereo_6slides.pdf)を測りましょう．

### 左画像と右画像の例を取得して表示します．
[Middlebury Stereo Datasets](http://vision.middlebury.edu/stereo/data/) の [2005 Datasets](http://vision.middlebury.edu/stereo/data/scenes2005/) を使います．

Q01: [Middlebury Stereo Vision Page](http://vision.middlebury.edu/stereo/)とは何か．どのような用途でどのようなデータがあるか．

In [ ]:
MB05_names = ["Art", "Books", "Dolls", "Laundry", "Moebius", "Reindeer"]
MB05_name = MB05_names[  4  ]  # choose 0 to 5
MB05_scales = [("FullSize", 1.0), ("HalfSize", 2.0), ("ThirdSize", 3.0)]
MB05_scale = MB05_scales[2]  # choose 0 to 2
zipURL = "http://vision.middlebury.edu/stereo/data/scenes2005/" + MB05_scale[0] + "/zip-2views/" + MB05_name +"-2views.zip"

print("Downloading the dataset " + MB05_name + " (" + MB05_scale[0] + ") ..")
!wget $zipURL --no-check-certificate --show-progress -q -O "/tmp/tmp.zip"

In [ ]:
import zipfile
import os

with zipfile.ZipFile("/tmp/tmp.zip", 'r') as f:
    f.extractall("/tmp")

import numpy
from PIL import Image

root_dir = "/tmp/" + MB05_name
left = numpy.asarray(Image.open(root_dir + '/view1.png'))
right = numpy.asarray(Image.open(root_dir + '/view5.png'))
disp1 = numpy.asarray(Image.open(root_dir + '/disp1.png')) / MB05_scale[1]
height, width, colors = left.shape
with open(root_dir + '/dmin.txt', 'r') as f:
    doffs = int(f.read()) / MB05_scale[1]

%matplotlib inline
import matplotlib.pyplot as plt
plt.figure(figsize=(15,4))
plt.subplot(1,3,1)
plt.imshow(left)
plt.subplot(1,3,2)
plt.imshow(right)
plt.subplot(1,3,3)
plt.imshow(disp1, cmap="gray")

### [OpenCV](https://opencv.org/)の[StereoBM](https://docs.opencv.org/4.1.2/dd/d53/tutorial_py_depthmap.html)を使ってみます．
Q02: [OpenCV](https://opencv.org/)とは何か．

Q03: 視差を推定するブロックマッチング（[stereoBM](https://docs.opencv.org/4.1.2/dd/d53/tutorial_py_depthmap.html)または[stereoSGBM](https://en.wikipedia.org/wiki/Semi-global_matching)）の仕組みを説明せよ．
> ヒント: 透視変換・逆透視変換と混同しないこと．

Q04: blockの値を変えると，視差の推定結果（disparity）はどのように変わるか．理由と共に答えよ．

In [ ]:
import cv2
disparities = 64
block = 15

# https://docs.opencv.org/4.1.2/dd/d53/tutorial_py_depthmap.html
#sbm = cv2.StereoBM_create(numDisparities=disparities, blockSize=block)

# https://en.wikipedia.org/wiki/Semi-global_matching
# https://core.ac.uk/download/pdf/11134866.pdf
sbm = cv2.StereoSGBM_create(numDisparities=disparities, blockSize=block, minDisparity=1)

cvleft = cv2.cvtColor(left, cv2.COLOR_RGB2GRAY)
cvright = cv2.cvtColor(right, cv2.COLOR_RGB2GRAY)
disparity = sbm.compute(cvleft, cvright) / 16
import matplotlib.pyplot as plt
plt.imshow(disparity, cmap="gray", vmin=0, vmax=disparities)

### 真値と比較してみましょう．
Q05: 物体の片側に大きな誤差が現れやすいのはなぜか．

In [ ]:
plt.figure(figsize=(15,4))
plt.subplot(1,3,1)
plt.title("Estimated")
plt.imshow(disparity, cmap="gray", vmin=0, vmax=disparities)
plt.subplot(1,3,2)
plt.title("Ground truth")
plt.imshow(disp1, cmap="gray", vmin=0, vmax=disparities)
plt.subplot(1,3,3)
plt.title("Error")
plt.imshow(numpy.abs(disparity-disp1), cmap="gray", vmin=0, vmax=disparities)

## ブロックマッチングの算法を自分で書いて確かめましょう．
1. [Depth from Stereo](https://github.com/IntelRealSense/librealsense/blob/master/doc/depth-from-stereo.md)をよく読んでください．
2. 仕組みを理解するために，画素毎の処理をfor文で実装します．[Depth from Stereo](https://github.com/IntelRealSense/librealsense/blob/master/doc/depth-from-stereo.md)のページの前半にある [SSD block-matching algorithm](https://github.com/IntelRealSense/librealsense/blob/master/doc/depth-from-stereo.md#stereoscopic-vision) を参考に下記のセルにコードを書き込んでください．
3. セルを実行してください．エラーがなければdisparityが表示されます．

### 注意：実行は数十秒～数分かかります．
Q06: OpenCV の関数を使って推定した視差とどこが異なるか．どのような工夫を施すと，StereoBM または StereoSGBM に近い推定結果や計算時間を達成できるか．

In [ ]:
block = 15
disparities = 64  # num of disparities to consider

xrange = lambda a,b: range(a,b)
disparity = numpy.zeros((height, width))

### https://github.com/IntelRealSense/librealsense/blob/master/doc/depth-from-stereo.md#stereoscopic-vision
### disparity[i, j]を計算する SSD (Sum of Squared Differences) block-matching algorithm
### （i,j,dのfor文）をここに書く．


import matplotlib.pyplot as plt
plt.imshow(disparity, cmap="gray", vmin=0, vmax=disparities)

In [ ]:
plt.figure(figsize=(15,4))
plt.subplot(1,3,1)
plt.title("Estimated")
plt.imshow(disparity, cmap="gray", vmin=0, vmax=disparities)
plt.subplot(1,3,2)
plt.title("Ground truth")
plt.imshow(disp1, cmap="gray", vmin=0, vmax=disparities)
plt.subplot(1,3,3)
plt.title("Error")
plt.imshow(numpy.abs(disparity-disp1), cmap="gray", vmin=0, vmax=disparities)

## 予習・復習
Q07: 視差を推定する方法には，ブロックマッチングの他にどのような手法があるか．[参考](https://www.dropbox.com/s/9mlomn6edk5rcpi/stereo.pdf?dl=1)

Q08: 本実験の例を，もし[能動的ステレオ（active stereo）](https://github.com/IntelRealSense/librealsense/blob/master/doc/depth-from-stereo.md#passive-vs-active-stereo)で実行したら，結果はどう異なるだろうか．本実験の例を示しながら説明せよ．

Q09: [Middlebury 2005 stereo datasets](http://vision.middlebury.edu/stereo/data/scenes2005/#description)の詳細では，カメラの焦点距離と基線の長さはそれぞれいくらと述べているか．単位と共に答えよ．

Q10: [Middlebury 2005 Stereo datasets](http://vision.middlebury.edu/stereo/data/scenes2005/)について，画素[i, j] の視差 d = disparity[i, j] から3次元座標(X,Y,Z)の深さZを計算する式を作成せよ．ただし，焦点距離をfocal_length，基線の長さを baseline とする．
> ヒント1: "dmin.txt" に書かれている値も使用する．

> ヒント2: [Middlebury 2014 stereo datasets](http://vision.middlebury.edu/stereo/data/scenes2014/#description)